In [79]:
import tweepy as tw
import pandas as pd
import re
import glob
import os
import schedule
import time
from textblob import TextBlob 
from datetime import datetime
import requests

In [80]:
class DataManager:
    def __init__(self):
        ##-------------------- twitter --------------------##
        self._url = "https://api.aiforthai.in.th/ssense"                     
        self._headers = {'Apikey': "0kFkiFLdf4TAyY3JeUT9WVnB5naP6SjW"}
        consumer_key = "EaFU9nJw2utR0lo2PUmJE3VZy"
        consumer_secret = "DsZuVw0tEl6GHhyK08tunsOE9ICSfwplEhRDMQwB8VIqngZ6i8"
        access_token = "759317188863897600-nuwQmcYfDX8lvdRyw2eCD6fMRMkLzzZ"
        access_token_secret = 'zFFc5OJywNMBrRAblI7kFV62ZTZPHfTU1Q5kZ1cKzUupD'
        auth = tw.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)

        self._api = tw.API(auth, wait_on_rate_limit=True)
        self.keys = []
        self.df = None
        self._start = 0

    def getSentiment(self,text):

        if TextBlob(text).sentiment.polarity > 0:
            return 'positive'
        elif TextBlob(text).sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

    def formatdatetime(self,column):
        self.df[column] = pd.to_datetime(self.df[column]).dt.strftime('%Y/%m/%d')
        self.df[column] = pd.to_datetime(self.df[column])
    
    def sortdf(self,columns):
        self.df.sort_values(by=columns,inplace=True)
        return self.df
        
    def searchkeys(self,keyword):
        self.keys = self.df['Keyword'].tolist()
        self.keys = list(set(self.keys))
        keyword = keyword.lower()
        if keyword in self.keys:
            return self.df.loc[self.df['Keyword']==keyword]
        else:
            print(f'{keyword} not in Database. Do you want to search?')
            Ans = str(input()).lower()                                                  #wait for GUI
            if Ans == 'yes':
                self.keys.append(keyword)
                # self.savedata()
                # return self.df.loc[self.df['Keyword']==keyword]
            else:
                pass

    def unionfile(self,filenames):              #type filename -> list
        for file in filenames:
            df1 = pd.read_csv(file)
            if self._start != 0:
                self.df = pd.concat([self.df,df1])
                self.df.drop_duplicates(keep='last',inplace=True)
            else:
                self.df = df1
                self._start += 1
        return self.df

    def getdate(self,since,until):  ####edit
        self.df.sort_values(by=['Time','Keyword'],inplace=True)
        mask = (self.df['Time']>=since) & (self.df['Time']<=until)
        return self.df.loc[mask]


In [81]:
filename = ['tweet_data_1932022.csv','tweet_data_2032022.csv','tweet_data_2132022_1.csv',
            'tweet_data_2232022.csv','tweet_data_2332022.csv','tweet_data_2432022.csv']

In [82]:
dm = DataManager()
dm.unionfile(filename)
dm.formatdatetime('Time')
#dm.sortdf(['Keyword','Time'])
#dm.getdate('2022-03-17','2022-03-23')
dm.searchkeys('shounen')

,Keyword,User,Retweet,Likes,Tweet,Time,Language,User Location,Hashtag,Polarity,Sentiment
2006,shounen,Shounen_w,0.0,1.0,@seongisdaddy never gonna happen ue,2022-03-19,NaN,NaN,NaN,NaN,NaN
2007,shounen,batkniqht,0.0,0.0,yeah they probably all read manga cuz they’re ...,2022-03-19,NaN,NaN,NaN,NaN,NaN
2008,shounen,lerinarin,0.0,0.0,Most of anime shounen in nutshell\nhttps://t.c...,2022-03-19,NaN,NaN,NaN,NaN,NaN
2009,shounen,AJShaw17,0.0,0.0,@BlueExe99 Where'd you find the clean version ...,2022-03-19,NaN,NaN,NaN,NaN,NaN
2010,shounen,voidgender_fizz,0.0,1.0,@jemibuni Me when my favorite character is a c...,2022-03-19,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
15679,shounen,Metavins1,1016.0,0.0,RT GodJiraslove WL GIVEAWAYWe are giving away ...,2022-03-24,en,HapeBeast HQ,[],neutral,0.0
15680,shounen,Renatta01760514,1016.0,0.0,RT GodJiraslove WL GIVEAWAYWe are giving away ...,2022-03-24,en,Indonesia,[],neutral,0.0
15681,shounen,ohkeyokey,1716.0,0.0,RT GodJiraslove WL GIVEAWAYWe are giving away ...,2022-03-24,en,NaN,[],neutral,0.0
15682,shounen,purplef30074728,148.0,0.0,RT n12smrai WL Giveaway for Shounen Ill be giv...,2022-03-24,en,NaN,[],neutral,0.0


In [83]:
import pandas as pd
import re
import glob
filename = ['tweet_data_2032022.csv','tweet_data_1932022.csv','tweet_data_2132022_1.csv','tweet_data_2232022.csv']
i = 0
for file in filename:
    df1 = pd.read_csv(file)
    if i != 0:
        df = pd.concat([df,df1])
        df.drop_duplicates(keep='last',inplace=True)
    else:
        df = df1
        i += 1

df['Time'] = pd.to_datetime(df['Time']).dt.strftime('%Y/%m/%d')
#df['Time'] = pd.to_datetime(df['Time']).dt.strftime('%d %m %Y')
df['Time'] = pd.to_datetime(df['Time'])
df.sort_values(by=['Time','Keyword'],inplace=True)

#time = list(set(df['Time'].tolist()))
mask = (df['Time']>='2022-03-19') & (df['Time']<='2022-03-21')
#mask = (df['Time']>='19-03-2022') & (df['Time']<='21-03-2022')
df.loc[mask]

,Keyword,User,Retweet,Likes,Tweet,Time,Language,User Location,Hashtag,Polarity,Sentiment
55,animation,SagarGa39569425,0.0,0.0,"The plot of ""Don't Mess With The Cultivation B...",2022-03-19,NaN,NaN,NaN,NaN,NaN
68,animation,Aarya_pimparkar,0.0,0.0,"I love the art style in ""Love After Marriage"" ...",2022-03-19,NaN,NaN,NaN,NaN,NaN
77,animation,jkhomezi,0.0,0.0,"You'll find companies looking for a ""Graphic D...",2022-03-19,NaN,NaN,NaN,NaN,NaN
165,animation,blazingwish66,0.0,0.0,"The plot of ""Martial Rebel"" is like a roller c...",2022-03-19,NaN,NaN,NaN,NaN,NaN
167,animation,DI0RDENKI,0.0,0.0,I think diluc should get a new idle animation ...,2022-03-19,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2975,อนิเมะแนะนำ,thanxxjk,20.0,0.0,RT todojamy,2022-03-21,th,NaN,['แนะนำอ'],positive,66.67
2976,อนิเมะแนะนำ,tinyreview2,37.0,0.0,RT bbrinkk bilibili,2022-03-21,th,ประเทศไทย,['สาววายถ'],positive,97.30
2978,อนิเมะแนะนำ,dalbluenim,27.0,0.0,RT primpytd Bilibili,2022-03-21,th,NaN,[],positive,75.00
2979,อนิเมะแนะนำ,btf_effectx,5.0,1.0,86 EIGHTYSIX Attack on Titan,2022-03-21,th,"she/her, th-en ♡",[],positive,83.33


In [12]:
import pandas as pd
import re
import glob
from nltk.corpus import stopwords
import langdetect
import requests
 
url = "https://api.aiforthai.in.th/ssense"
headers = {'Apikey': "0kFkiFLdf4TAyY3JeUT9WVnB5naP6SjW"}

en_stops = set(stopwords.words('english'))
filename = ['tweet_data_2432022.csv','tweet_data_2332022.csv','tweet_data_2232022.csv']
i = 0
for file in filename:
    df1 = pd.read_csv(file)
    if i != 0:
        df = pd.concat([df,df1])
        df.drop_duplicates(keep='last',inplace=True)
    else:
        df = df1
        i += 1

keys = (df['Keyword']=='ต่างโลก') | (df['Keyword']=='อนิเมะ')
dff = df.loc[keys].sort_values(by=['Language','Keyword'])
word = {}

for i in dff['Tweet']:
    if langdetect.detect(i) != 'th':
        allwords = i.split()
        for w in allwords: 
            if w not in en_stops:
                if w in word:
                    word[w] += 1
                else:
                    word[w] = 1
    else:
        data = {'text':i}
        response = requests.post(url, data=data, headers=headers)
        allwords = response.json()['preprocess']['keyword']
        for w in allwords: 
            if w not in en_stops:
                if w in word:
                    word[w] += 1
                else:
                    word[w] = 1
del word['RT']
sortword = sorted(word.items(),key=lambda x:x[1],reverse=True)
for i in sortword[:5]:
    print(i)

20638    EUREKA SEVEN GOOD NIGHT SLEEP TIGHT YOUNG LOVE...
15904                                           RT etalork
15905                                         RT Algiz7375
15906                                         RT Suparak1a
15908                       RT NeptheduckNovel OpenThisSky
                               ...                        
2930                                            RT lermloe
2931                                            RT lermloe
2932                                       RT tingjapanese
2933                                            RT lermloe
2934                                       RT tingjapanese
Name: Tweet, Length: 3198, dtype: object

In [ ]:
from nltk.corpus import stopwords
en_stops = set(stopwords.words('english'))

all_words = ['There', 'is', 'a', 'tree','near','the','river']
for word in all_words: 
    if word not in en_stops:
        print(word)

There
tree
near
river


In [ ]:
import requests
 
url = "https://api.aiforthai.in.th/ssense"
 
text = 'สาขานี้พนักงานน่ารักให้บริการดี'
 
data = {'text':text}
 
headers = {
    'Apikey': "0kFkiFLdf4TAyY3JeUT9WVnB5naP6SjW"
    }
 
response = requests.post(url, data=data, headers=headers)
 
print(response.json())
response.json()['preprocess']['keyword']

{'sentiment': {'score': '88.89', 'polarity-neg': False, 'polarity-pos': True, 'polarity': 'positive'}, 'preprocess': {'input': 'สาขานี้พนักงานน่ารักให้บริการดี', 'neg': [], 'pos': ['น่ารัก', 'ให้บริการดี'], 'segmented': ['สาขา', 'นี้', 'พนักงาน', 'น่ารัก', 'ให้บริการดี'], 'keyword': ['สาขา', 'พนักงาน']}, 'alert': [], 'comparative': [], 'associative': [{'ent-pos': [], 'polarity-neg': False, 'endIndex': 20, 'polarity-pos': True, 'beginIndex': 7, 'text': 'พนักงานน่ารัก', 'ent-neg': [], 'asp': ['พนักงาน']}], 'intention': {'request': '0', 'sentiment': '88.89', 'question': '0', 'announcement': '0'}}


['สาขา', 'พนักงาน']

In [ ]:
key = list(set(df['Keyword'].tolist()))
key

['harem',
 'อนิเมะ',
 'school life',
 'japan animation',
 'disney animation',
 'from novel',
 '2d animation',
 'ต่างโลก',
 'anime',
 'anime comedy',
 'อนิเมะแนะนำ',
 'animation',
 'slice of life anime',
 'mappa',
 'sport anime',
 '3d',
 'seinen',
 'shoujo',
 'bl anime',
 '3d animation',
 'attack on titan',
 'pixar',
 'fantasy anime',
 'from manga',
 'shounen',
 'anime romance',
 'school life anime',
 'shounen ai',
 'animation studio']